In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import itertools
import argparse
import sys
import time
from sklearn import preprocessing
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'


In [ ]:
## Preprocessing of data
# Function to load data

def get_ghg_data():
    def splitDataset(totaldata, train = 0.8, seed = 123, normalize = True):
        # seed
        np.random.seed(seed)
        # number
        numdata = totaldata.shape[0]
        numtrain = int(numdata*train)
        numtest = numdata - numtrain
        # index
        index = np.arange(numdata)
        np.random.shuffle(index)
        # shuffle
        totaldata = totaldata[index,:,:]
        
        totaldata = totaldata.reshape(numdata, -1)
        totaldata = totaldata.T
        totaldata = totaldata/np.linalg.norm(totaldata, axis=0)
        totaldata = totaldata.T
        
        # split
        traindata = totaldata[:numtrain, :]
        testdata = totaldata[numtrain:, :]
        # split X, Y
        # train
        X_train = traindata[:,:-1]
        Y_train = traindata[:,-1].reshape(-1)

        # test
        X_test = testdata[:,:-1]
        Y_test = testdata[:,-1].reshape(-1)
        return X_train, Y_train, X_test, Y_test

    data = []
    data_folder = '/home/zehang/Downloads/dataset/ghg_data/'
    filelist = os.listdir(data_folder)
    for file in filelist:
        data_single = np.genfromtxt(data_folder+file,dtype=np.float)
        data.append(data_single)
        
    data = np.array(data)
    X_train, Y_train, X_test, Y_test = splitDataset(data)
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
X_train, X_test, y_train, y_test = get_ghg_data()
print("X,y types: {} {}".format(type(X_train), type(y_train)))
print("X size {}".format(X_train.shape))
print("Y size {}".format(y_train.shape))

In [ ]:
# Sigmoid function
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x)) 
    if derivative:
        return sigm * (1. - sigm)
    return sigm

# Define weights initialization
def initialize_w(N, d):
    return 2*np.random.random((N,d)) - 1

# Fill in feed forward propagation
def feed_forward_propagation(X, y, w_1, w_2, w_3, lmbda):
    # Fill in
    # X (N,d)
    # w_1 (d,h)
    # w_2 (h,g)
    # w_3 (g,1)
    N,d = X.shape
    layer_0 = X # (N,d)
    layer_1 = sigmoid(np.dot(layer_0, w_1)) # (N, h)
    layer_2 = sigmoid(np.dot(layer_1, w_2)) # (N, g)
    layer_3 = np.dot(layer_2, w_3) # (N, 1)
    
    return layer_0, layer_1, layer_2, layer_3
def back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3, lmbda):
    N = y.shape[0]
    y = y.reshape((-1,1))
    layer_3_delta = np.zeros_like(w_3) # (g,1)
    layer_2_delta = np.zeros_like(w_2) # (h, g)
    layer_1_delta = np.zeros_like(w_1) # (d, h)
    layer_3_delta = 2 * np.dot(layer_2.T, (layer_3 - y)) # (g,1)
    # print("np.dot(w_3,(layer_3-y)) shape ", np.dot(w_3,(layer_3-y).T).shape)
    # print(" sigmoid(np.dot(layer_1,w_2), derivative=True).T shape", sigmoid(np.dot(layer_1,w_2), derivative=True).T.shape)
    dJ_dl2 = 2 * np.dot(w_3,(layer_3-y).T) # # (g,N)
    dl2_ds2 = sigmoid(np.dot(layer_1,w_2), derivative=True).T # (g,N)

    layer_2_delta  = np.dot(dJ_dl2 * dl2_ds2, layer_1).T    
    # layer_2_delta = 2 * np.dot(np.dot(w_3,(layer_3-y).T)*sigmoid(np.dot(layer_1,w_2), derivative=True).T, layer_1).T
    ds2_dl1 = w_2 # (h,g)
    dl1_ds1 =  sigmoid(np.dot(layer_0,w_1), derivative=True).T # (h,N)
    ds1_dw1 = layer_0 # (N,d)

    layer_1_delta = np.dot(np.dot(ds2_dl1, dJ_dl2 * dl2_ds2) * dl1_ds1, ds1_dw1).T # (d,h)
    return layer_1_delta/N, layer_2_delta/N, layer_3_delta/N

def back_propagation_blocklayer(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3, lmbda, activelayer = 3):
    N = y.shape[0]
    y = y.reshape((-1,1))
    layer_3_delta = np.zeros_like(w_3) # (g,1)
    layer_2_delta = np.zeros_like(w_2) # (h, g)
    layer_1_delta = np.zeros_like(w_1) # (d, h)
    layer_3_delta = 2 * np.dot(layer_2.T, (layer_3 - y)) # (g,1)
    
    if activelayer == 3:
        return layer_1_delta/N, layer_2_delta/N, layer_3_delta/N
    
    # print("np.dot(w_3,(layer_3-y)) shape ", np.dot(w_3,(layer_3-y).T).shape)
    # print(" sigmoid(np.dot(layer_1,w_2), derivative=True).T shape", sigmoid(np.dot(layer_1,w_2), derivative=True).T.shape)
    dJ_dl2 = 2 * np.dot(w_3,(layer_3-y).T) # # (g,N)
    dl2_ds2 = sigmoid(np.dot(layer_1,w_2), derivative=True).T # (g,N)

    layer_2_delta  = np.dot(dJ_dl2 * dl2_ds2, layer_1).T    
    if activelayer == 2:
        return layer_1_delta/N, layer_2_delta/N, layer_3_delta/N
    
    # layer_2_delta = 2 * np.dot(np.dot(w_3,(layer_3-y).T)*sigmoid(np.dot(layer_1,w_2), derivative=True).T, layer_1).T
    ds2_dl1 = w_2 # (h,g)
    dl1_ds1 =  sigmoid(np.dot(layer_0,w_1), derivative=True).T # (h,N)
    ds1_dw1 = layer_0 # (N,d)

    layer_1_delta = np.dot(np.dot(ds2_dl1, dJ_dl2 * dl2_ds2) * dl1_ds1, ds1_dw1).T # (d,h)
    if activelayer == 1:
        return layer_1_delta/N, layer_2_delta/N, layer_3_delta/N

# Cost function
def cost(X, y, w_1, w_2, w_3, lmbda):
    N, d = X.shape
    a1,a2,a3,a4 = feed_forward_propagation(X,y,w_1,w_2,w_3,lmbda)

#     return np.linalg.norm(a4[:,0] - y,2) ** 2 / N + lmbda * (np.linalg.norm(w_1)**2 + np.linalg.norm(w_2)**2 + np.linalg.norm(w_3)**2)
    return np.linalg.norm(a4[:,0] - y,2) ** 2 / N

# Define SGD
def SGD(X, y, w_1, w_2, w_3, lmbda, learning_rate, batch_size, iterations):
    # Complete here:
    loss_lst = []
    time_lst = []
    start = time.time()
    y = y.reshape((-1,1))
    for i in range(iterations):
        loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        loss_lst.append(loss)
        
        randomInd = np.arange(X.shape[0])
        np.random.shuffle(randomInd)
        randomInd = randomInd[:batch_size]
        layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X[randomInd,:],y[randomInd,:],w_1,w_2,w_3,lmbda)
        layer_1_delta, layer_2_delta, layer_3_delta = back_propagation(y[randomInd,:], w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3, lmbda)
        
        g = (np.linalg.norm(layer_1_delta) + np.linalg.norm(layer_2_delta) + np.linalg.norm(layer_3_delta))/3
        if i%20==0:
            print("g is ", g, loss)   
        if (g <= epsilon):
#             loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
            time_lst.append(time.time()-start)
            print("converge, break! current i: ", i, loss)
            break
        
        w_1 = w_1 - learning_rate*layer_1_delta
        w_2 = w_2 - learning_rate*layer_2_delta
        w_3 = w_3 - learning_rate*layer_3_delta
    
        
#         loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        
#         if i%20==0:
#             print(i,loss)
        time_lst.append(time.time()-start)
    return w_1, w_2, w_3, loss_lst, time_lst

# Define SVRG here:
def SVRG(X, y, w_1, w_2, w_3, lmbda, learning_rate, T, batch_size, iterations):
    # Complete here:
    y = y.reshape((-1,1))
    N = X.shape[0]
    loss_lst = []
    time_lst = []
    start = time.time()
    for i in range(iterations):
        loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        loss_lst.append(loss)
        # compute all gradient and store
        layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X,y,w_1,w_2,w_3,lmbda)
        layer_1_delta, layer_2_delta, layer_3_delta = back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3, lmbda)
            
        
        g = (np.linalg.norm(layer_1_delta) + np.linalg.norm(layer_2_delta) + np.linalg.norm(layer_3_delta))/3
        if i%20==0:
            print("g is ", g, loss)   
        if (g <= epsilon):
#             loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)  
            time_lst.append(time.time()-start)
            
            print("converge, break! current i: ", i, loss)
            break
        
        # initialize the w_previous
        # w_previous = w.copy()
        w_1_previous, w_2_previous, w_3_previous = w_1.copy(), w_2.copy(), w_3.copy()
        for t in range(T//batch_size):
            # random sample
            # randomInd = int(np.random.rand() * N)
            randomInd = np.arange(N)
            np.random.shuffle(randomInd)
            randomInd = randomInd[:batch_size]
            # randomInd = np.random.randint(0,N)
            layer_0_p1, layer_1_p1, layer_2_p1, layer_3_p1 = feed_forward_propagation(X[randomInd,:],y[randomInd,:],w_1_previous,w_2_previous,w_3_previous,lmbda)
            layer_1_delta_p1, layer_2_delta_p1, layer_3_delta_p1 = back_propagation(y[randomInd,:], w_1_previous, w_2_previous, w_3_previous, layer_0_p1, layer_1_p1, layer_2_p1, layer_3_p1, lmbda)

            layer_0_p2, layer_1_p2, layer_2_p2, layer_3_p2 = feed_forward_propagation(X[randomInd,:],y[randomInd,:],w_1,w_2,w_3,lmbda)
            layer_1_delta_p2, layer_2_delta_p2, layer_3_delta_p2 = back_propagation(y[randomInd,:], w_1, w_2, w_3, layer_0_p2, layer_1_p2, layer_2_p2, layer_3_p2, lmbda)
            
            # calculate the update term
            # part1 = function_gradient_vectorization(x[:,randomInd], y[:,randomInd], w_previous, lambda_, gradclip = gradclip)
            # part2 = function_gradient_vectorization(x[:,randomInd], y[:,randomInd], w, lambda_, gradclip = gradclip)
            # part3 = g

            w_1_previous = w_1_previous - learning_rate * (layer_1_delta_p1 - layer_1_delta_p2 + layer_1_delta)
            w_2_previous = w_2_previous - learning_rate * (layer_2_delta_p1 - layer_2_delta_p2 + layer_2_delta)
            w_3_previous = w_3_previous - learning_rate * (layer_3_delta_p1 - layer_3_delta_p2 + layer_3_delta)

            # w_previous = w_previous - alpha * (part1 - part2 + part3)
            
        # w = w_previous
        
        w_1, w_2, w_3 = w_1_previous, w_2_previous, w_3_previous
        time_lst.append(time.time()-start)
        
#         loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
#         if i%20==0:
#             print(i,loss)
    
    return w_1, w_2, w_3, loss_lst, time_lst

# Define GD here:
def GD(X, y, w_1,w_2,w_3, lmbda, learning_rate, iterations):
    N = X.shape[0]
    loss_lst = []
    time_lst = []
    start = time.time()
    for i in range(iterations):
        loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        loss_lst.append(loss)
        
        layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
        layer_1_delta, layer_2_delta, layer_3_delta = back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda)
        
        g = (np.linalg.norm(layer_1_delta) + np.linalg.norm(layer_2_delta) + np.linalg.norm(layer_3_delta))/3
        if i%20==0:
            print("g is ", g, loss)   
        if (g <= epsilon):
#             loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
            time_lst.append(time.time()-start)
            
            print("converge, break! current i: ", i, loss)
            break
#         w_1 = w_1 -  learning_rate * layer_1_delta + (lmbda / N * w_1)
#         w_2 = w_2 - learning_rate * layer_2_delta + (lmbda / N * w_2)
#         w_3 = w_3 - learning_rate * layer_3_delta + (lmbda / N * w_3)
        w_1 = w_1 -  learning_rate * layer_1_delta
        w_2 = w_2 - learning_rate * layer_2_delta
        w_3 = w_3 - learning_rate * layer_3_delta
#         loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
#         if i%20==0:
#             print(i,loss)
        time_lst.append(time.time()-start)

    return w_1, w_2, w_3, loss_lst, time_lst

# Define projected GD here:
def PGD(X, y, w_1,w_2,w_3, lmbda, learning_rate, iterations, noise = None):
    # Complete here:
    N = X.shape[0]
    loss_lst = []
    time_lst = []
    start = time.time()
    for i in range(iterations):
        loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        loss_lst.append(loss)
        
        layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
        layer_1_delta, layer_2_delta, layer_3_delta = back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda)
        
        g = (np.linalg.norm(layer_1_delta) + np.linalg.norm(layer_2_delta) + np.linalg.norm(layer_3_delta))/3
        if i%20==0:
            print("g is ", g, loss)   
        if (g <= epsilon):
#             loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
            time_lst.append(time.time()-start)
            
            print("converge, break! current i: ", i,loss)
            break
            
        if noise == None:
            # the noise level is determined by the delta
            noise = []
            noise.append((np.random.random((w_1.shape)) * 2 - 1) * 0.0001)
            noise.append((np.random.random((w_2.shape)) * 2 - 1) * 0.0001)
            noise.append((np.random.random((w_3.shape)) * 2 - 1) * 0.0001)
        
        w_1 = w_1 -  learning_rate * layer_1_delta + noise[0]
        w_2 = w_2 - learning_rate * layer_2_delta + noise[1]
        w_3 = w_3 - learning_rate * layer_3_delta + noise[2]
#         loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        
#         if i%20==0:
#             print(i,loss)
        time_lst.append(time.time()-start)

    return w_1, w_2, w_3, loss_lst, time_lst

# Define BCD here:
def BCD(X, y, w_1,w_2,w_3, lmbda, learning_rate, iterations, strategy = 0, blockrate = 0.5):
    loss_lst = []
    time_lst = []
    start = time.time()
    # Complete here:
    # 2 strategies:
    # a. randomly select weight by a random mask (random coordinate selection)
    # b. we train the layers asynchronously to archieve the goal of blocking coordinate (cyclic update rule) 
    for i in range(iterations):
        loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
        loss_lst.append(loss)
        
        layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
        layer_1_delta, layer_2_delta, layer_3_delta = back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda)
        
        g = (np.linalg.norm(layer_1_delta) + np.linalg.norm(layer_2_delta) + np.linalg.norm(layer_3_delta))/3
        if i%20==0:
            print("g is ", g, loss)   
        if (g <= epsilon):
            time_lst.append(time.time()-start)
            
#             loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
            print("converge, break! current i: ", i,loss)
            break
            
        if strategy == 0:
            layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
            # generate random mask for each layer
            l1_mask = np.random.choice([1, 0], size=layer_1_delta.shape, p=[blockrate, 1 - blockrate])
            l2_mask = np.random.choice([1, 0], size=layer_2_delta.shape, p=[blockrate, 1 - blockrate])
            l3_mask = np.random.choice([1, 0], size=layer_3_delta.shape, p=[blockrate, 1 - blockrate])
            w_1 = w_1 - learning_rate * layer_1_delta * l1_mask
            w_2 = w_2 - learning_rate * layer_2_delta * l2_mask
            w_3 = w_3 - learning_rate * layer_3_delta * l3_mask
        else:
            # cyclic
            # layer 1
            layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
            layer_1_delta, _, _ = back_propagation_blocklayer(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda, 1)
            w_1 = w_1 - learning_rate * layer_1_delta
            # layer 2
            layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
            _, layer_2_delta, _ = back_propagation_blocklayer(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda, 2)
            w_2 = w_2 - learning_rate * layer_2_delta
            # layer 3
            layer_0, layer_1, layer_2, layer_3 = feed_forward_propagation(X, y, w_1,w_2,w_3,lmbda)
            _, _, layer_3_delta = back_propagation_blocklayer(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3,lmbda, 3)
            w_3 = w_3 - learning_rate * layer_3_delta
#         loss = cost(X_train, y_train, w_1, w_2, w_3, lmbda)
#         if i%20==0:
#             print(i,loss)
        time_lst.append(time.time()-start)

    return w_1, w_2, w_3, loss_lst, time_lst

In [ ]:
# Should be a hyperparameter that you tune, not an argument - Fill in the values
lmbda =0.01
w_size = 50
lr = 0.002
iterations = 5000 # 100
T = 2000
batch_size = 100
epsilon = 0.05

# Initialize weights
w_1 = initialize_w(X_train.shape[1], w_size)

w_2 = initialize_w(w_size,w_size)

w_3 = initialize_w(w_size, 1)

print("GD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_gd = time.time()
w_1_star,w_2_star,w_3_star,loss_gd,time_gd = GD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_gd = time.time()
print("Training time for GD: ", end_gd-start_gd)

print("PGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_pgd = time.time()
w_1_star,w_2_star,w_3_star,loss_pgd,time_pgd = PGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_pgd = time.time()
print("Training time for PGD: ", end_pgd-start_pgd)


print("SGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_sgd = time.time()
w_1_star,w_2_star,w_3_star,loss_sgd,time_sgd = SGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, batch_size, iterations)
end_sgd = time.time()
print("Training time for SGD: ", end_sgd-start_sgd)



print("SVRG\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_svrg = time.time()
w_1_star1,w_2_star1,w_3_star1,loss_svrg,time_svrg = SVRG(X_train, y_train, w_1, w_2, w_3, lmbda, lr, T, batch_size, iterations)
end_svrg = time.time()
print("Training time for SVRG: ", end_svrg-start_svrg)

print("BCD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
strategy = 0 # 0 for random block mask, 1 for layer block
blockrate = 0.5
start_bcd = time.time()
w_1_star,w_2_star,w_3_star ,loss_bcd,time_bcd = BCD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations, strategy, blockrate)
end_bcd = time.time()
print("Training time for BCD: ", end_bcd-start_bcd)


In [ ]:
# Should be a hyperparameter that you tune, not an argument - Fill in the values
lmbda =0.01
w_size = 50
lr = 0.001
iterations = 5000 # 100
T = 2000
batch_size = 100
epsilon = 0.05

# Initialize weights
w_1 = initialize_w(X_train.shape[1], w_size)

w_2 = initialize_w(w_size,w_size)

w_3 = initialize_w(w_size, 1)

print("GD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_gd = time.time()
w_1_star,w_2_star,w_3_star, loss_gd1,time_gd1 = GD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_gd = time.time()
print("Training time for GD: ", end_gd-start_gd)

print("PGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_pgd = time.time()
w_1_star,w_2_star,w_3_star, loss_pgd1,time_pgd1 = PGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_pgd = time.time()
print("Training time for PGD: ", end_pgd-start_pgd)


print("SGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_sgd = time.time()
w_1_star,w_2_star,w_3_star, loss_sgd1,time_sgd1 = SGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, batch_size, iterations)
end_sgd = time.time()
print("Training time for SGD: ", end_sgd-start_sgd)



print("SVRG\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_svrg = time.time()
w_1_star1,w_2_star1,w_3_star1, loss_svrg1,time_svrg1 = SVRG(X_train, y_train, w_1, w_2, w_3, lmbda, lr, T, batch_size, iterations)
end_svrg = time.time()
print("Training time for SVRG: ", end_svrg-start_svrg)

print("BCD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
strategy = 0 # 0 for random block mask, 1 for layer block
blockrate = 0.5
start_bcd = time.time()
w_1_star,w_2_star,w_3_star, loss_bcd1,time_bcd1 = BCD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations, strategy, blockrate)
end_bcd = time.time()
print("Training time for BCD: ", end_bcd-start_bcd)


In [ ]:
'''

    1

'''


# Should be a hyperparameter that you tune, not an argument - Fill in the values
lmbda =0.1
w_size = 50
lr = 0.001
iterations = 5000 # 100
T = 2000
batch_size = 100
epsilon = 0.05

# Initialize weights
w_1 = initialize_w(X_train.shape[1], w_size)

w_2 = initialize_w(w_size,w_size)

w_3 = initialize_w(w_size, 1)

print("GD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_gd = time.time()
w_1_star,w_2_star,w_3_star, loss_gd2, time_gd2 = GD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_gd = time.time()
print("Training time for GD: ", end_gd-start_gd)

print("PGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_pgd = time.time()
w_1_star,w_2_star,w_3_star, loss_pgd2,time_pgd2 = PGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_pgd = time.time()
print("Training time for PGD: ", end_pgd-start_pgd)


print("SGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_sgd = time.time()
w_1_star,w_2_star,w_3_star, loss_sgd2,time_sgd2 = SGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, batch_size, iterations)
end_sgd = time.time()
print("Training time for SGD: ", end_sgd-start_sgd)



print("SVRG\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_svrg = time.time()
w_1_star1,w_2_star1,w_3_star2, loss_svrg2,time_svrg2 = SVRG(X_train, y_train, w_1, w_2, w_3, lmbda, lr, T, batch_size, iterations)
end_svrg = time.time()
print("Training time for SVRG: ", end_svrg-start_svrg)

print("BCD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
strategy = 0 # 0 for random block mask, 1 for layer block
blockrate = 0.5
start_bcd = time.time()
w_1_star,w_2_star,w_3_star, loss_bcd2,time_bcd2 = BCD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations, strategy, blockrate)
end_bcd = time.time()
print("Training time for BCD: ", end_bcd-start_bcd)


In [ ]:
'''

    Same parameter
    
'''


# Should be a hyperparameter that you tune, not an argument - Fill in the values
lmbda =0.1
w_size = 50
lr = 0.001
iterations = 5000 # 100
T = 2000
batch_size = 100
epsilon = 0.05

# Initialize weights
w_1 = initialize_w(X_train.shape[1], w_size)

w_2 = initialize_w(w_size,w_size)

w_3 = initialize_w(w_size, 1)

print("GD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_gd = time.time()
w_1_star,w_2_star,w_3_star, loss_gd2, time_gd2 = GD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_gd = time.time()
print("Training time for GD: ", end_gd-start_gd)

print("PGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_pgd = time.time()
w_1_star,w_2_star,w_3_star, loss_pgd2,time_pgd2 = PGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations)
end_pgd = time.time()
print("Training time for PGD: ", end_pgd-start_pgd)


print("SGD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_sgd = time.time()
w_1_star,w_2_star,w_3_star, loss_sgd2,time_sgd2 = SGD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, batch_size, iterations)
end_sgd = time.time()
print("Training time for SGD: ", end_sgd-start_sgd)



print("SVRG\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
start_svrg = time.time()
w_1_star1,w_2_star1,w_3_star2, loss_svrg2,time_svrg2 = SVRG(X_train, y_train, w_1, w_2, w_3, lmbda, lr, T, batch_size, iterations)
end_svrg = time.time()
print("Training time for SVRG: ", end_svrg-start_svrg)

print("BCD\tinitial loss is :", cost(X_train, y_train, w_1, w_2, w_3, lmbda))
strategy = 0 # 0 for random block mask, 1 for layer block
blockrate = 0.5
start_bcd = time.time()
w_1_star,w_2_star,w_3_star, loss_bcd2,time_bcd2 = BCD(X_train, y_train, w_1, w_2, w_3, lmbda, lr, iterations, strategy, blockrate)
end_bcd = time.time()
print("Training time for BCD: ", end_bcd-start_bcd)


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16, 8))

# Plot results
ax[0].legend(loc="upper right")
ax[0].set_xlabel(r"Iteration", fontsize=16)
ax[0].set_ylabel("Loss", fontsize=16)
ax[0].set_title("CA3 - Training a deep neural network for the ghg Dataset")
ax[0].set_ylim(ymin=0, ymax=5)
ax[0].plot(np.arange(len(loss_gd)), loss_gd, label="GD")
ax[0].plot(np.arange(len(loss_pgd[:120])), loss_pgd[:120], label="PGD")
ax[0].plot(np.arange(len(loss_sgd)), loss_sgd, label="SGD")
ax[0].plot(np.arange(len(loss_svrg)), loss_svrg, label="SVRG")
ax[0].plot(np.arange(len(loss_bcd)), loss_bcd, label="BCD")
ax[0].legend()

ax[1].legend(loc="upper right")
ax[1].set_xlabel(r"Time [s]", fontsize=16)
ax[1].set_ylabel("Loss", fontsize=16)
ax[1].set_ylim(ymin=0, ymax=5)
ax[1].plot(time_gd, loss_gd, label="GD")
ax[1].plot(time_pgd[:120], loss_pgd[:120], label="PGD")
ax[1].plot(time_sgd, loss_sgd, label="SGD")
ax[1].plot(time_svrg, loss_svrg, label="SVRG")
ax[1].plot(time_bcd, loss_bcd, label="BCD")
ax[1].legend()

plt.savefig("ghg-power-Loss-Iter.png")

In [ ]:
print(len(time_svrg1))
print(len(loss_svrg1))


fig, ax = plt.subplots(2, 1, figsize=(16, 8))

# Plot results
ax[0].legend(loc="upper right")
ax[0].set_xlabel(r"Iteration", fontsize=16)
ax[0].set_ylabel("Loss", fontsize=16)
ax[0].set_title("CA3 - Training a deep neural network for the ghg Dataset")
ax[0].set_ylim(ymin=0, ymax=5)
ax[0].plot(np.arange(len(loss_gd1)), loss_gd1, label="GD")
ax[0].plot(np.arange(len(loss_pgd1[:280])), loss_pgd1[:280], label="PGD")
ax[0].plot(np.arange(len(loss_sgd1)), loss_sgd1, label="SGD")
ax[0].plot(np.arange(len(loss_svrg1[:10])), loss_svrg1[:10], label="SVRG")
ax[0].plot(np.arange(len(loss_bcd1)), loss_bcd1, label="BCD")
ax[0].legend()

ax[1].legend(loc="upper right")
ax[1].set_xlabel(r"Time [s]", fontsize=16)
ax[1].set_ylabel("Loss", fontsize=16)
ax[1].set_ylim(ymin=0, ymax=5)
ax[1].plot(time_gd1, loss_gd1, label="GD")
ax[1].plot(time_pgd1[:280], loss_pgd1[:280], label="PGD")
ax[1].plot(time_sgd1, loss_sgd1, label="SGD")
ax[1].plot(time_svrg1, loss_svrg1[:10], label="SVRG")
ax[1].plot(time_bcd1, loss_bcd1, label="BCD")
ax[1].legend()

plt.savefig("ghg-power-Loss-Iter1.png")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16, 8))

# Plot results
ax[0].legend(loc="upper right")
ax[0].set_xlabel(r"Iteration", fontsize=16)
ax[0].set_ylabel("Loss", fontsize=16)
ax[0].set_title("CA3 - Training a deep neural network for the ghg Dataset")
ax[0].set_ylim(ymin=0, ymax=5)
ax[0].plot(np.arange(len(loss_gd2)), loss_gd2, label="GD")
ax[0].plot(np.arange(len(loss_pgd2[:114])), loss_pgd2[:114], label="PGD")
ax[0].plot(np.arange(len(loss_sgd2)), loss_sgd2, label="SGD")
ax[0].plot(np.arange(len(loss_svrg1)), loss_svrg1, label="SVRG")
ax[0].plot(np.arange(len(loss_bcd2)), loss_bcd2, label="BCD")
ax[0].legend()

ax[1].legend(loc="upper right")
ax[1].set_xlabel(r"Time [s]", fontsize=16)
ax[1].set_ylabel("Loss", fontsize=16)
ax[1].set_ylim(ymin=0, ymax=5)
ax[1].plot(time_gd2, loss_gd2, label="GD")
ax[1].plot(time_pgd2[:114], loss_pgd2[:114], label="PGD")
ax[1].plot(time_sgd2, loss_sgd2, label="SGD")
ax[1].plot(time_svrg2, loss_svrg2, label="SVRG")
ax[1].plot(time_bcd2, loss_bcd2, label="BCD")
ax[1].legend()

plt.savefig("ghg-power-Loss-Iter2.png")